In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from IPython.display import clear_output
import time
import catboost
import re
import optuna
import json
import sys
sys.path.append('../..')
import main

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (FunctionTransformer, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, PowerTransformer,
                                   OneHotEncoder)
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, StratifiedShuffleSplit
from sklearn.base import BaseEstimator, TransformerMixin

col_names = []
with open('../data/Faults27x7_var','r') as f:
    for line in f:
        col_names.append(line.strip())
        
train_org = pd.read_csv('../data/train.csv')
test_org = pd.read_csv('../data/test.csv')
org_data = pd.read_csv('../data/Faults.NNA', delimiter='\s', engine='python', names=col_names)

X = train_org.drop(['id','Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps','Other_Faults'], axis=1)
pastry = train_org['Pastry'].copy()
z_scratch = train_org['Z_Scratch'].copy()
k_scatch = train_org['K_Scatch'].copy()
stains = train_org['Stains'].copy()
dirtiness = train_org['Dirtiness'].copy()
bumps = train_org['Bumps'].copy()
other_faults = train_org['Other_Faults'].copy()

ys = [pastry, z_scratch, k_scatch, stains, dirtiness, bumps, other_faults]
y_names = ['pastry', 'z_scratch', 'k_scatch', 'stains', 'dirtiness', 'bumps', 'other_faults']

class KMeansTransformer(BaseEstimator,TransformerMixin):
    def __init__(self, n_clusters):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=self.n_clusters, n_init=10, random_state=0)
        
    def fit(self,X, y=None):
        self.kmeans.fit(X)
        return self
        
    def transform(self,X):
        labels = self.kmeans.predict(X)
        return np.c_[X, labels]

In [2]:
y_params_dict = {}
for y_nm in y_names:
    y_params_dict[y_nm] = None

In [3]:
y_scores = {}

In [4]:
for y,y_name in zip(ys,y_names):
    
    print(f'trial: {y_name}')
    
    def objective_lgb(trial):
    
        cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        params = dict(
            n_estimators = trial.suggest_int('n_estimators',100,500),
            max_depth = trial.suggest_int('max_depth',2,64),
            num_leaves = trial.suggest_int('num_leaves',2,128),
            learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
            min_child_samples = trial.suggest_int('min_child_samples',2,500),
            min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
            subsample = trial.suggest_float('subsample', 0.33,0.85),
            colsample_bytree = trial.suggest_float('colsample_bylevel',0.33,0.7),
            reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
            reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
        )
        
        lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary', verbose=-1,**params)
        
        pipe = Pipeline(
            steps = [
                ('scaler', StandardScaler()),
                ('model', lgbc)
            ]
        )
        
        score = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
        return score
    
    study_lgb = optuna.create_study(direction='maximize')
    
    study_lgb.optimize(objective_lgb, n_trials=1000, n_jobs=-1, show_progress_bar=True)
    
    best_params = study_lgb.best_params
    y_params_dict[y_name] = best_params
    
    best_score = study_lgb.best_value
    y_scores[y_name] = best_score
    
    time.sleep(2)
    clear_output()

In [5]:
description = 'lgbc vanilla with hpt'
main.saver(y_params_dict,y_scores,description)

In [6]:
y_params_dict = {}
for y_nm in y_names:
    y_params_dict[y_nm] = {}
    for i in [4,5,7]:
        if i ==1:
            continue
        y_params_dict[y_nm][i] = None

In [7]:
y_scores = {}
for yns in y_names:
    y_scores[yns] = {}
    for n in [4,5,7]:
        y_scores[yns][n] = None

In [8]:
for y,yn in zip(ys,y_names):
    for n,i in enumerate([4,5,7]):
        
        print(f'trial: {yn}(n_cluster={i})')
        
        def objective_lgb(trial):
    
            cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
            params = dict(
                n_estimators = trial.suggest_int('n_estimators',100,500),
                max_depth = trial.suggest_int('max_depth',2,64),
                max_leaves = trial.suggest_int('max_leaves',2,128),
                learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
                gamma = trial.suggest_float('gamma',0.001,10),
                min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
                subsample = trial.suggest_float('subsample', 0.33,0.85),
                colsample_bytree = trial.suggest_float('colsample_bytree',0.33,0.7),
                reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
                reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
            )
            
            lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary', verbose=-1,**params)
            
            pipe = Pipeline(
                steps = [
                    ('scaler', StandardScaler()),
                    ('kmeans', KMeansTransformer(n_clusters=i)),
                    ('model', lgbc)
                ]
            )
            
            score = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
            return score
        
        study_lgb = optuna.create_study(direction='maximize')
        
        study_lgb.optimize(objective_lgb, n_trials=1000, n_jobs=-1, show_progress_bar=True)
        
        best_params = study_lgb.best_params
        y_params_dict[yn][i] = best_params
        
        best_score = study_lgb.best_value
        y_scores[yn][i] = best_score
        
        time.sleep(2)
        clear_output()
        
description = 'lgbc experiment with varying n_clusters(4,5,7) along with hpt'
main.saver(y_params_dict,y_scores,description)

In [9]:
time.sleep(300)

In [10]:
for y,yn in zip(ys,y_names):
    for n,i in enumerate([2,3]):
        
        print(f'trial: {yn}(n_cluster={i})')
        
        def objective_lgb(trial):
    
            cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
            params = dict(
                n_estimators = trial.suggest_int('n_estimators',100,500),
                max_depth = trial.suggest_int('max_depth',2,64),
                max_leaves = trial.suggest_int('max_leaves',2,128),
                learning_rate = trial.suggest_float('learning_rate',0.001,0.3),
                gamma = trial.suggest_float('gamma',0.001,10),
                min_child_weight = trial.suggest_float('min_child_weight', 0.01,10),
                subsample = trial.suggest_float('subsample', 0.33,0.85),
                colsample_bytree = trial.suggest_float('colsample_bytree',0.33,0.7),
                reg_alpha=trial.suggest_float('reg_alpha', 0.001, 0.1),
                reg_lambda = trial.suggest_float('reg_lambda', 0.001,0.1)
            )
            
            lgbc = lgb.LGBMClassifier(random_state= 0, objective='binary', verbose=-1,**params)
            
            pipe = Pipeline(
                steps = [
                    ('scaler', StandardScaler()),
                    ('kmeans', KMeansTransformer(n_clusters=i)),
                    ('model', lgbc)
                ]
            )
            
            score = np.mean(cross_val_score(pipe, X,y, scoring='roc_auc', cv= cvo))
            return score
        
        study_lgb = optuna.create_study(direction='maximize')
        
        study_lgb.optimize(objective_lgb, n_trials=1000, n_jobs=-1, show_progress_bar=True)
        
        best_params = study_lgb.best_params
        y_params_dict[yn][i] = best_params
        
        best_score = study_lgb.best_value
        y_scores[yn][i] = best_score
        
        time.sleep(2)
        clear_output()
        
description = 'lgbc experiment with varying n_clusters(2,3) along with hpt'
main.saver(y_params_dict,y_scores,description)

In [31]:
with open('../../artifacts/12_Mar_06_54_23.json','r') as file:
    rd = json.load(file)

In [32]:
rd

{'pastry': 0.8719188684326825,
 'z_scratch': 0.9613491379485734,
 'k_scatch': 0.9861826041592829,
 'stains': 0.9929364569004854,
 'dirtiness': 0.8984424720696227,
 'bumps': 0.8117495758112815,
 'other_faults': 0.7091654213801943}

In [33]:
fs = 0
for key in rd.keys():
    
    fs += rd[key]
    
print(f'\nfinal_score: {fs/7}')


final_score: 0.8902492195288746


In [34]:
with open('../../artifacts/12_Mar_06_54_19.json','r') as file:
    pd = json.load(file)

In [35]:
pastry_params = pd['pastry']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [36]:
z_scratch_params = pd['z_scratch']

z_scratch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**z_scratch_params))
    ]
)
z_scratch_m.fit(X,z_scratch)
z_scratch_pred = z_scratch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [37]:
k_scatch_params = pd['k_scatch']

k_scatch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**k_scatch_params))
    ]
)
k_scatch_m.fit(X,k_scatch)
k_scatch_pred = k_scatch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [38]:
stains_params = pd['stains']

stains_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**stains_params))
    ]
)
stains_m.fit(X,stains)
stains_pred = stains_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [39]:
dirtiness_params = pd['dirtiness']

dirtiness_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**dirtiness_params))
    ]
)
dirtiness_m.fit(X,dirtiness)
dirtiness_pred = dirtiness_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [40]:
bumps_params = pd['bumps']

bumps_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**bumps_params))
    ]
)
bumps_m.fit(X,bumps)
bumps_pred = bumps_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [41]:
other_faults_params = pd['other_faults']

other_faults_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**other_faults_params))
    ]
)
other_faults_m.fit(X,other_faults)
other_faults_pred = other_faults_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [44]:
sub = pd.DataFrame({'id':test_org['id'].copy(), 'Pastry':pastry_pred, 'Z_Scratch':z_scratch_pred, 'K_Scatch':k_scatch_pred,
                    'Stains':stains_pred, 'Dirtiness':dirtiness_pred, 'Bumps':bumps_pred, 'Other_Faults':other_faults_pred})

In [45]:
sub.head()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.552399,0.001176,0.002196,0.000092,0.010662,0.161874,0.340980
1,19220,0.299751,0.011878,0.000943,0.000153,0.182748,0.171236,0.326938
2,19221,0.002521,0.048136,0.037964,0.000734,0.005654,0.297688,0.477086
3,19222,0.155694,0.001702,0.000196,0.000986,0.011084,0.377820,0.437101
4,19223,0.003227,0.002123,0.000201,0.004021,0.007892,0.629521,0.399463


In [46]:
sub.to_csv('../submissions/m12_1.csv', index=False)

In [ ]:
# requires to load necessary file

In [28]:
fs = 0
for key in rd.keys():
    val = 0
    nc = 0
    for sub_key in rd[key].keys():
        if rd[key][sub_key] > val:
            val = rd[key][sub_key]
            nc = int(sub_key)
    fs += val
    print(f'{key}, n_clusters:{nc}, score:{val}')
    
print(f'\nfinal_score: {fs/7}')

pastry, n_clusters:4, score:0.8720316370550407
z_scratch, n_clusters:5, score:0.9614436879626707
k_scatch, n_clusters:7, score:0.9860581348821281
stains, n_clusters:7, score:0.9929668328934336
dirtiness, n_clusters:2, score:0.8964052730066661
bumps, n_clusters:2, score:0.8117091433569099
other_faults, n_clusters:2, score:0.7092812461693652

final_score: 0.8899851364751734


In [12]:
with open('../../artifacts/12_Mar_19_59_03.json','r') as file:
    pd = json.load(file)

In [13]:
pastry_params = pd['pastry']['4']

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=4)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [15]:
z_scratch_params = pd['z_scratch']['5']

z_scratch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**z_scratch_params))
    ]
)
z_scratch_m.fit(X,z_scratch)
z_scratch_pred = z_scratch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [16]:
k_scatch_params = pd['k_scatch']['7']

k_scatch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('KMeans', KMeansTransformer(n_clusters=7)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**k_scatch_params))
    ]
)
k_scatch_m.fit(X,k_scatch)
k_scatch_pred = k_scatch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [18]:
stains_params = pd['stains']['7']

stains_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(7)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**stains_params))
    ]
)
stains_m.fit(X,stains)
stains_pred = stains_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [20]:
dirtiness_params = pd['dirtiness']['2']

dirtiness_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(2)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**dirtiness_params))
    ]
)
dirtiness_m.fit(X,dirtiness)
dirtiness_pred = dirtiness_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [21]:
bumps_params = pd['bumps']['2']

bumps_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(2)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**bumps_params))
    ]
)
bumps_m.fit(X,bumps)
bumps_pred = bumps_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [22]:
other_faults_params = pd['other_faults']['2']

other_faults_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(2)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**other_faults_params))
    ]
)
other_faults_m.fit(X,other_faults)
other_faults_pred = other_faults_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [26]:
sub = pd.DataFrame({'id':test_org['id'].copy(), 'Pastry':pastry_pred, 'Z_Scratch':z_scratch_pred, 'K_Scatch':k_scatch_pred,
                    'Stains':stains_pred, 'Dirtiness':dirtiness_pred, 'Bumps':bumps_pred, 'Other_Faults':other_faults_pred})

In [27]:
sub.head()

,id,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,19219,0.481416,0.000935,0.008414,0.000023,0.020232,0.154150,0.317981
1,19220,0.374900,0.019936,0.006365,0.000080,0.133880,0.160983,0.321465
2,19221,0.001870,0.047187,0.040209,0.000177,0.005786,0.322317,0.489446
3,19222,0.147363,0.001075,0.001337,0.001100,0.012126,0.359708,0.432672
4,19223,0.003465,0.001280,0.002112,0.003120,0.007647,0.614681,0.390724


In [30]:
sub.to_csv('../submissions/m12.csv', index=False)

In [20]:
#1
pastry_params = {'n_estimators': 280,
 'max_depth': 9,
 'num_leaves': 18,
 'learning_rate': 0.02148602951813924,
 'min_child_samples': 341,
 'min_child_weight': 6.361004402846124,
 'subsample': 0.49585801139053814,
 'colsample_bylevel': 0.35930633871573076,
 'reg_alpha': 0.06447122051496705,
 'reg_lambda': 0.016497887759421646}

pastry_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**pastry_params))
    ]
)
pastry_m.fit(X,pastry)
pastry_pred = pastry_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [21]:
z_scratch_params = {'n_estimators': 296,
 'max_depth': 3,
 'num_leaves': 74,
 'learning_rate': 0.03879534308586422,
 'min_child_samples': 298,
 'min_child_weight': 2.4215929637347218,
 'subsample': 0.47753292687280846,
 'colsample_bylevel': 0.35906419523398436,
 'reg_alpha': 0.057670762264904175,
 'reg_lambda': 0.0427161308925542}

z_scratch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**z_scratch_params))
    ]
)
z_scratch_m.fit(X,z_scratch)
z_scratch_pred = z_scratch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [22]:
k_scatch_params = {'n_estimators': 271,
 'max_depth': 29,
 'num_leaves': 118,
 'learning_rate': 0.02560845874325205,
 'min_child_samples': 449,
 'min_child_weight': 1.802873155395419,
 'subsample': 0.7902749625886126,
 'colsample_bylevel': 0.3433292920431201,
 'reg_alpha': 0.047379430530416024,
 'reg_lambda': 0.010007517965380167}

k_scatch_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**k_scatch_params))
    ]
)
k_scatch_m.fit(X,k_scatch)
k_scatch_pred = k_scatch_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [23]:
stains_params = {'n_estimators': 266,
 'max_depth': 5,
 'num_leaves': 75,
 'learning_rate': 0.027151102431374634,
 'min_child_samples': 170,
 'min_child_weight': 1.024782187730875,
 'subsample': 0.5803990695479041,
 'colsample_bylevel': 0.6394813204481101,
 'reg_alpha': 0.09477440109735585,
 'reg_lambda': 0.05246070547701958}

stains_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**stains_params))
    ]
)
stains_m.fit(X,stains)
stains_pred = stains_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [24]:
dirtiness_params = {'n_estimators': 100,
 'max_depth': 13,
 'num_leaves': 62,
 'learning_rate': 0.04095550709994935,
 'min_child_samples': 331,
 'min_child_weight': 0.1953843881250878,
 'subsample': 0.4674944721594767,
 'colsample_bylevel': 0.33072889947558426,
 'reg_alpha': 0.07067954909592045,
 'reg_lambda': 0.0017098736746246962}

dirtiness_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('kmeans', KMeansTransformer(n_clusters=5)),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**dirtiness_params))
    ]
)
dirtiness_m.fit(X,dirtiness)
dirtiness_pred = dirtiness_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [25]:
bumps_params = y_params_dict['bumps']

bumps_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**bumps_params))
    ]
)
bumps_m.fit(X,bumps)
bumps_pred = bumps_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [26]:
other_faults_params = y_params_dict['other_faults']

other_faults_m = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', lgb.LGBMClassifier(random_state= 0, objective='binary', verbose = -1,**other_faults_params))
    ]
)
other_faults_m.fit(X,other_faults)
other_faults_pred = other_faults_m.predict_proba(test_org.drop(['id'], axis=1))[:,1]

In [27]:
sub = pd.DataFrame({'id':test_org['id'].copy(), 'Pastry':pastry_pred, 'Z_Scratch':z_scratch_pred, 'K_Scatch':k_scatch_pred,
                    'Stains':stains_pred, 'Dirtiness':dirtiness_pred, 'Bumps':bumps_pred, 'Other_Faults':other_faults_pred})

In [28]:
sub.to_csv('../submissions/m5_1.csv', index=False)

In [29]:
(0.8721124548680992 + 0.9613490481775984 + 0.9860986294660549 + 0.9931795666268013 + 0.8967476825823397 + 0.871617374898657 + 0.8716315587132517)/7

0.9218194736189717